In [1]:
export PYTHONPATH="${PYTHONPATH}:/data/exp/kostya/ukr_g2p"
export PYTHONPATH="${PYTHONPATH}:/data/exp/kostya/ukroASR"
UKROASR_ROOT=/data/exp/kostya/ukroASR
# export PYTHONIOENCODING=utf-8

In [2]:
cd $UKROASR_ROOT

In [44]:
cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh

In [3]:
train_cmd=run.pl

In [4]:
work_dir=/data/exp/kostya/ukroASR

In [5]:
data=$work_dir/data

In [6]:
exp=$work_dir/exp

In [43]:
dvc run -y \
-d $data/raw_data/train.csv -d $UKROASR_ROOT/gen_data_dir.py \
-o $data/train/text -o $data/train/utt2spk -o $data/train/segments -o $data/train/spk2utt -o $data/train/wav.scp \
-f train_data.dvc \
python3.6 $UKROASR_ROOT/gen_data_dir.py $data/raw_data/train.csv $data/train

Running command:                                                        
	python3.6 /data/exp/kostya/ukroASR/gen_data_dir.py /data/exp/kostya/ukroASR/data/raw_data/train.csv /data/exp/kostya/ukroASR/data/train
/bin/sh: 18: ./cmd.sh: [[: not found
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/utt2spk is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/spk2utt is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/text is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/segments is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/wav.scp is not in sorted order or not unique, sorting it
fix_data_dir.sh: kept all 10811 utterances.
fix_data_dir.sh: old files are kept in /data/exp/kostya/ukroASR/data/train/.backup
                       

In [44]:
dvc run -y \
-d $data/raw_data/dev.csv -d $UKROASR_ROOT/gen_data_dir.py \
-o $data/dev/text -o $data/dev/utt2spk -o $data/dev/segments -o $data/dev/spk2utt -o $data/dev/wav.scp \
-f dev_data.dvc \
python3.6 $UKROASR_ROOT/gen_data_dir.py $data/raw_data/dev.csv $data/dev

Running command:                                                        
	python3.6 /data/exp/kostya/ukroASR/gen_data_dir.py /data/exp/kostya/ukroASR/data/raw_data/dev.csv /data/exp/kostya/ukroASR/data/dev
/bin/sh: 18: ./cmd.sh: [[: not found
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/utt2spk is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/spk2utt is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/text is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/segments is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/wav.scp is not in sorted order or not unique, sorting it
fix_data_dir.sh: kept all 879 utterances.
fix_data_dir.sh: old files are kept in /data/exp/kostya/ukroASR/data/dev/.backup
                                         

In [45]:
git add train_data.dvc dev_data.dvc data/train/.gitignore data/dev/.gitignore

In [46]:
git commit -m "Update kaldi's train and dev dirs"

[master ae6a831] Update kaldi's train and dev dirs
 3 files changed, 16 insertions(+), 6 deletions(-)
 create mode 100644 data/dev/.gitignore
 create mode 100644 data/train/.gitignore


In [12]:
# for split in train dev
# do
# cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/fix_data_dir.sh $data/$split
# done

utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/utt2spk is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/text is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/segments is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/train/wav.scp is not in sorted order or not unique, sorting it
fix_data_dir.sh: kept all 10813 utterances.
fix_data_dir.sh: old files are kept in /data/exp/kostya/ukroASR/data/train/.backup
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/utt2spk is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/text is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/exp/kostya/ukroASR/data/dev/segments is not in sorted order or not unique, sorting it
utils/fix_data_dir.sh: file /data/ex

In [17]:
# preparing dict directory

In [18]:
# additional_text_corpus=/data/exp/kostya/ukr_asr/text_corpus/text_plane

In [19]:
# create tempral text file out of corpus text and train_text

In [41]:
# tmp_text_corpus=/data/exp/kostya/ukr_asr/text_corpus/text

In [42]:
# cp $data/dev/text $tmp_text_corpus

In [43]:
# cut -f 2- -d ' ' $data/train/text >> $tmp_text_corpus

In [9]:
# # generating vocab file
# python3.6 $work_dir/tools/gen_vocab.py -t $data/train/text \
#                                        -v $data/local/dict/vocab

In [10]:
# # generating lexicon file
# python3.6 $work_dir/tools/gen_lexicon.py -v $data/local/dict/vocab \
#                                          -o $data/local/dict/lexicon.txt

Failed on: b\xd0\xbe\xd0\xb4\xd0\xb2i\xd1\x80\xd0\xba\xd0\xb0\xd0\xbc\xd0\xb8
Failed on: b\xd1\x81\xd0\xb8\xd0\xb4i\xd1\x82\xd0\xb8
Failed on: b\xd0\xb1i\xd0\xba
Failed on: b\xd0\xbf\xd0\xbe\xd0\xbbi\xd0\xb7
Failed on: b\xd0\xbf\xd0\xbb\xd0\xb5\xd1\x87i
Failed on: b\xd0\xbc\xd0\xbb\xd0\xb8\xd0\xbd\xd0\xbe\xd0\xb2i
Failed on: b\xd0\xb2i\xd0\xba\xd0\xbd\xd0\xb0
Failed on: b\xd0\xb1\xd0\xb8\xd1\x81\xd1\x82\xd1\x80\xd0\xb8\xd0\xbdi
Failed on: b\xd1\x81\xd1\x82i\xd0\xbd\xd0\xb0\xd1\x85
Failed on: b\xd0\xb4i\xd0\xb6\xd1\x83
Failed on: bmal
Failed on: b\xd0\xbd\xd0\xb5\xd0\xb2i\xd1\x81\xd1\x82\xd0\xba\xd1\x83
Failed on: b\xd0\xbf\xd0\xbe\xd1\x87\xd0\xbe\xd1\x80\xd0\xbdi\xd0\xbb\xd0\xbe
Failed on: b\xd1\x84\xd0\xb8\xd0\xbb\xd0\xb8\xd0\xbf\xd0\xbf\xd1\x8b\xd1\x87
Failed on: bthat
Failed on: b\xd0\xbfi\xd0\xb4\xd0\xba\xd0\xb8\xd0\xb4\xd0\xb0\xd0\xb9
Failed on: b\xd0\xbf\xd0\xbe\xd0\xbd\xd0\xb5\xd0\xb4i\xd0\xbb\xd0\xbe\xd0\xba
Failed on: b\xd0\xb4i\xd0\xb2\xd1\x87\xd0\xb8\xd0\xbdi
Failed on: b\xd

Failed on: b\xd0\xbfi\xd0\xb7\xd0\xbd\xd0\xb0\xd0\xb2
Failed on: b\xd0\xb1i\xd1\x94\xd0\xb2\xd1\x86\xd1\x8f\xd1\x85
Failed on: bplace
Failed on: b\xd0\xb4i\xd0\xb2\xd1\x81\xd1\x8f
Failed on: b1848
Failed on: b\xd0\xb7\xd0\xbe\xd0\xb2\xd1\x81i\xd0\xbc
Failed on: b\xd1\x82i\xd1\x82\xd0\xba\xd0\xb0
Failed on: b\xd0\xb2\xd0\xb5\xd1\x80\xd1\x81i
Failed on: b\xd0\xb2i\xd0\xb4\xd1\x8c\xd0\xbc\xd0\xbe
Failed on: b\xd0\xb1i\xd0\xbb\xd1\x8c\xd1\x88\xd0\xb0
Failed on: b\xd0\xb2\xd0\xb8\xd0\xb1i\xd0\xb3\xd0\xbb\xd0\xb8
Failed on: b\xd1\x85\xd1\x80\xd0\xb5\xd1\x81\xd1\x82i
Failed on: b\xd1\x81i\xd0\xbb
Failed on: b\xd0\xbfi\xd1\x88\xd0\xbb\xd0\xb0
Failed on: b\xd0\xba\xd0\xbb\xd1\x8e\xd1\x87i
Failed on: b\xd0\xba\xd0\xbe\xd0\xbci\xd1\x80
Failed on: b\xd0\xb2\xd0\xb1i\xd0\xb3\xd0\xbb\xd0\xb0
Failed on: b\xd0\xb1\xd1\x83\xd1\x80\xd1\x8f\xd0\xbai\xd0\xb2
Failed on: b\xd0\xb2i\xd1\x97
Failed on: b\xd0\xb3\xd0\xbe\xd1\x80\xd0\xb4i
Failed on: b\xd0\xb7\xd0\xb0\xd0\xb1\xd0\xb0\xd0\xb3\xd0\xb0\xd1\x82i\xd1

Failed on: b\xd1\x81\xd0\xb2i\xd1\x82\xd0\xb8\xd0\xbb\xd0\xbe
Failed on: b\xd0\xb1\xd0\xbbi\xd0\xb4\xd0\xb8\xd0\xbc
Failed on: b\xd0\xb3\xd1\x80\xd0\xbe\xd1\x88i
Failed on: b\xd0\xb2\xd0\xb5\xd1\x81i\xd0\xbb\xd0\xbb\xd1\x8e
Failed on: b\xd0\xb3\xd1\x83\xd1\x81\xd1\x82i-\xd0\xbf\xd1\x80\xd0\xb5\xd0\xb3\xd1\x83\xd1\x81\xd1\x82i
Failed on: b\xd0\xb7\xd0\xb0\xd0\xb4\xd0\xbdi\xd0\xbc\xd0\xb8
Failed on: b\xd0\xbf\xd1\x80\xd0\xb8\xd0\xb4\xd0\xb0\xd0\xb2i\xd1\x82\xd1\x8c
Failed on: b\xd0\xb7\xd0\xb0\xd0\xbbi\xd0\xb7\xd0\xbd\xd0\xb8\xd0\xbc\xd0\xb8
Failed on: b\xd0\xbf\xd0\xbe\xd1\x81\xd0\xb8\xd0\xb4i\xd0\xbb\xd0\xb0
Failed on: b\xd1\x81\xd0\xbbi\xd0\xbfi
Failed on: b\xd0\xb1\xd0\xb0\xd0\xb1i
Failed on: b\xd1\x80\xd0\xbe\xd0\xb7\xd0\xbf\xd1\x80i\xd0\xb3
Failed on: beighty
Failed on: bi\xd0\xbd\xd1\x88\xd0\xb0
Failed on: b\xd0\xbe\xd0\xb4i\xd1\x80\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c
Failed on: b\xd0\xba\xd0\xb0\xd1\x80\xd0\xbf\xd0\xbe\xd0\xb2i\xd0\xb9
Failed on: b\xd1\x81\xd0\xbe\xd1\x82\xd0\xbdi
Fa

Failed on: b\xd0\xbd\xd0\xb0\xd0\xb2\xd0\xba\xd0\xbe\xd0\xbbi\xd1\x88\xd0\xba\xd0\xb8
Failed on: bveuve
Failed on: b\xd0\xb1\xd1\x83\xd0\xb3\xd0\xb0\xd1\x94\xd0\xb2i
Failed on: b\xd1\x81\xd1\x82\xd1\x83\xd0\xbb\xd0\xb5\xd0\xbdi
Failed on: b\xd0\xb7\xd0\xb0\xd0\xba\xd1\x83\xd1\x86i\xd1\x97
Failed on: b\xd0\xbf\xd0\xbe\xd1\x80i\xd0\xb6\xd0\xb5
Failed on: b\xd0\xbfi\xd0\xb4\xd0\xb4\xd1\x83\xd1\x80\xd0\xb8
Failed on: b\xd0\xbf\xd0\xb0\xd0\xb2\xd0\xb8\xd0\xbdi
Failed on: b\xd0\xb4\xd1\x80i\xd0\xb1\xd0\xbd\xd0\xb5
Failed on: b\xd0\xb2i\xd0\xbd\xd0\xba\xd0\xbe\xd0\xbc
Failed on: bsaint-sophie
Failed on: b\xd0\xbe\xd1\x81\xd0\xb2i\xd1\x82\xd0\xb8\xd0\xbb\xd0\xbe\xd1\x81\xd1\x8f
Failed on: b\xd0\xb4\xd0\xbe\xd0\xbbi
Failed on: b\xd1\x82\xd0\xb5\xd0\xbf\xd0\xbbi\xd0\xb9
Failed on: byes
Failed on: b\xd1\x81\xd0\xbci\xd1\x82\xd0\xbd\xd0\xb8\xd0\xba
Failed on: b\xd0\xbci\xd1\x88\xd0\xba\xd0\xb8
Failed on: b\xd0\xbc\xd0\xbe\xd0\xbd\xd0\xb0\xd1\x81\xd1\x82\xd0\xb8\xd1\x80i
Failed on: b\xd0\xb1\xd0\xb

Failed on: b\xd1\x86i\xd0\xba\xd0\xb0\xd0\xb2\xd0\xb8\xd0\xb9
Failed on: b\xd0\xb2\xd0\xb3\xd0\xbb\xd1\x8f\xd0\xb4i\xd0\xb2
Failed on: b\xd0\xb1\xd0\xbe\xd0\xb3\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb4\xd0\xb8\xd1\x86i
Failed on: byou
Failed on: b\xd0\xb7\xd0\xbd\xd0\xb0\xd0\xb2i\xd1\x81\xd0\xbdi\xd0\xb2
Failed on: b\xd0\xb6i\xd0\xbd\xd0\xba\xd0\xbe
Failed on: b26
Failed on: b\xd0\xb4i\xd0\xbb\xd0\xb0
Failed on: b\xd0\xb4i\xd0\xbb
Failed on: b\xd0\xba\xd0\xbe\xd0\xb6\xd0\xbdi\xd0\xb9
Failed on: b\xd0\xbe\xd1\x81\xd0\xbci\xd1\x85\xd0\xbd\xd1\x83\xd0\xb2\xd1\x81\xd1\x8f
Failed on: b29
Failed on: b\xd1\x81i\xd0\xbc
Failed on: b\xd0\xbdi\xd1\x81
Failed on: b\xd0\xbbi\xd1\x82
Failed on: b\xd0\xbf\xd0\xbe\xd0\xbci\xd0\xb6
Failed on: bcrise
Failed on: b\xd0\xb1\xd0\xb5\xd0\xb7\xd1\x83\xd0\xbc\xd0\xbd\xd1\x8b\xd1\x85
Failed on: b\xd0\xbfi\xd0\xb2\xd0\xb4\xd0\xb5\xd0\xbd\xd0\xbd\xd0\xb5
Failed on: b\xd0\xb1i\xd0\xbb\xd1\x8c\xd1\x88\xd0\xb8\xd0\xbc
Failed on: b\xd0\xbe\xd0\xb4i\xd1\x80\xd0\xb2\xd1\x83
Fa

...

Failed on: b\xd0\xb5\xd0\xb1\xd1\x80\xd1\x8f\xd0\xbd\xd1\x8b\xd0\xb5
Failed on: b\xd0\xbdi\xd0\xb6
Failed on: b\xd1\x81\xd0\xb2\xd0\xb5\xd0\xba\xd1\x80\xd1\x83\xd1\x81i
Failed on: b\xd1\x81\xd0\xbci\xd1\x82\xd1\x82\xd1\x8f
Failed on: b\xd0\xbci\xd1\x86\xd0\xbd\xd0\xb8\xd0\xbc
Failed on: b\xd0\xbe\xd0\xb1\xd1\x80\xd0\xb0\xd0\xb7i\xd0\xb2
Failed on: b\xd1\x81\xd0\xbf\xd0\xbe\xd0\xb2i\xd0\xb4\xd0\xb0\xd1\x82\xd0\xb8\xd1\x81\xd1\x8c
Failed on: b\xd0\xb0\xd0\xbd\xd0\xb4\xd1\x80i\xd1\x97\xd0\xb2\xd1\x81\xd1\x8c\xd0\xba\xd0\xb0
Failed on: b\xd1\x83\xd0\xb2i\xd0\xb2
Failed on: b\xd1\x81\xd0\xbf\xd0\xbe\xd0\xb4i\xd0\xb2\xd0\xb0\xd0\xb2\xd1\x81\xd1\x8f
Failed on: b\xd0\xbf\xd0\xbe\xd0\xb7\xd0\xbci\xd1\x82\xd0\xb0\xd0\xb9
Failed on: b\xd1\x81\xd0\xbfi\xd0\xb2\xd1\x83
Failed on: b\xd1\x81\xd0\xb2i\xd1\x82\xd0\xbe\xd0\xbc
Failed on: b\xd0\xbd\xd0\xb5\xd0\xb1\xd0\xbe\xd0\xb6i\xd0\xb2
Failed on: b\xd0\xbb\xd0\xb0\xd0\xb2i
Failed on: b\xd0\xbf\xd0\xbe\xd1\x81\xd1\x82\xd0\xb5\xd1\x80i\xd0\xb3
Failed on

Failed on: b\xd0\xb3\xd1\x80\xd1\x83\xd0\xbd\xd1\x82i
Failed on: b\xd0\xb1\xd0\xbb\xd0\xb8\xd1\x81\xd0\xba\xd1\x83\xd1\x87i
Failed on: b\xd0\xbf\xd1\x80\xd0\xbe\xd0\xbci\xd0\xbd\xd1\x8c
Failed on: b\xd0\xba\xd0\xbe\xd1\x80i\xd0\xb2
Failed on: b\xd0\xbf\xd1\x80\xd1\x83\xd0\xb4\xd0\xbai
Failed on: b\xd0\xbf\xd0\xb5\xd1\x80\xd0\xb5\xd0\xbci\xd1\x80\xd1\x8f\xd0\xb2\xd1\x88\xd0\xb8
Failed on: b\xd0\xb2\xd0\xbe\xd0\xb7i
Failed on: b\xd0\xb7\xd0\xb1\xd0\xbe\xd0\xb6\xd0\xb5\xd0\xb2\xd0\xbe\xd0\xbbi\xd0\xb2
Failed on: b\xd1\x83\xd0\xba\xd1\x83\xd0\xbfi
Failed on: b\xd1\x81\xd0\xbdi\xd0\xb3
Failed on: b\xd0\xbe\xd0\xb3i\xd1\x80\xd0\xbai\xd0\xb2
Failed on: b\xd0\xb2i\xd0\xbd\xd0\xb5\xd1\x86\xd1\x8c
Failed on: b\xd0\xbf\xd1\x80\xd0\xbe\xd0\xb2i\xd1\x82\xd1\x80\xd0\xb8\xd1\x82\xd1\x8c
Failed on: b\xd0\xb3\xd0\xbe\xd0\xbbi
Failed on: b\xd0\xbe\xd1\x85\xd1\x80i\xd0\xbc\xd0\xba\xd1\x83
Failed on: b\xd0\xbe\xd1\x81\xd0\xbci\xd1\x85
Failed on: b\xd1\x85\xd0\xbe\xd1\x82i\xd0\xbb\xd0\xb0
Failed on: b\xd0\

Failed on: b\xd1\x81\xd1\x80i\xd0\xb1\xd0\xbd\xd0\xbe\xd1\x97
Failed on: b\xd1\x87\xd1\x83\xd0\xb6i\xd0\xb9
Failed on: boukraina
Failed on: binvite
Failed on: bcomes
Failed on: b\xd0\xb7i\xd0\xb9\xd0\xb4\xd0\xb8
Failed on: b\xd0\xb1i\xd0\xb9\xd1\x81\xd1\x8f
Failed on: b\xd0\xb7\xd0\xb0\xd1\x80\xd0\xbe\xd0\xb1i\xd1\x82\xd0\xbai\xd0\xb2
Failed on: b\xd0\xb7\xd0\xb0\xd0\xb7\xd0\xb5\xd0\xbb\xd0\xb5\xd0\xbdi\xd0\xb2
Failed on: b\xd0\xbbi\xd1\x81\xd0\xb0\xd1\x85
Failed on: b\xd0\xbf\xd0\xbe\xd0\xb2\xd0\xbdi\xd0\xb9
Failed on: b\xd1\x80i\xd0\xb4
Failed on: b\xd0\xbe\xd1\x80\xd0\xb0\xd0\xbd\xd1\x86i
Failed on: b\xd0\xbf\xd0\xbe\xd0\xb2i\xd1\x8f\xd0\xbb\xd0\xbe
Failed on: b\xd0\xbdi\xd0\xb1\xd0\xb8
Failed on: b\xd0\xb4\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xbdi
Failed on: b\xd0\xbd\xd0\xb5\xd0\xb2i\xd1\x81\xd1\x82\xd0\xba\xd0\xbe
Failed on: b\xd0\xb2\xd0\xb8\xd0\xb1i\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe
Failed on: b\xd0\xbfi\xd0\xb2\xd0\xbdi
Failed on: b\xd1\x85\xd0\xbc\xd0\xb0\xd1\x80\xd1\x86

Failed on: b\xd0\xb1i\xd0\xbbi\xd0\xbb\xd0\xb0
Failed on: b\xd1\x80\xd0\xbe\xd0\xb7\xd0\xb3\xd0\xbdi\xd0\xb2\xd0\xb0\xd0\xbd\xd1\x83
Failed on: b\xd1\x85\xd1\x83\xd1\x81\xd1\x82\xd0\xb8\xd0\xbdi
Failed on: b\xd0\xb1i\xd0\xb3\xd0\xb0\xd1\x94
Failed on: b\xd1\x87\xd0\xb5\xd1\x80\xd0\xb2\xd0\xbe\xd0\xbdi\xd0\xbb\xd0\xb0
Failed on: b\xd0\xb7\xd0\xb0\xd0\xbci\xd0\xbd\xd1\x8f\xd0\xbb\xd0\xb8
Failed on: b\xd1\x81\xd0\xb5\xd0\xbc\xd0\xb8\xd0\xb3\xd0\xbe\xd1\x80\xd1\x81\xd1\x8c\xd0\xbai
Failed on: b\xd1\x80\xd0\xbe\xd0\xb7\xd0\xb2i\xd1\x88\xd1\x83\xd1\x8e\xd1\x87\xd0\xb8
Failed on: b60
Failed on: b\xd0\xbci\xd1\x81\xd1\x82\xd0\xb0
Failed on: b\xd0\xb4i\xd0\xb1\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0
Failed on: b\xd0\xbbi\xd1\x81\xd0\xbe\xd0\xb2\xd0\xbe\xd1\x8e
Failed on: b\xd1\x81\xd1\x8b\xd0\xb5\xd0\xbb\xd0\xb5-\xd0\xb3\xd0\xb5
Failed on: b\xd0\xbe\xd0\xb4\xd0\xb3\xd0\xbe\xd0\xb2i\xd0\xbb\xd0\xb0\xd1\x81\xd1\x8c
Failed on: b\xd0\xbe\xd0\xb3i\xd1\x80\xd0\xbe\xd1\x87\xd0\xba\xd0\xb8
Failed on: b\xd0\xbc

Failed on: b\xd0\xbe\xd1\x81\xd0\xbbi\xd0\xbf
Failed on: b\xd0\xb2\xd0\xb5\xd1\x87i\xd1\x80\xd0\xbd\xd1\x94
Failed on: b\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xb4\xd0\xb8\xd0\xb2i
Failed on: b\xd0\xbf\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xb2\xd0\xb8\xd0\xbdi
Failed on: b\xd0\xb7\xd0\xbci\xd1\x8e\xd0\xba\xd0\xb8
Failed on: bi\xd0\xb3\xd1\x83\xd0\xbc\xd0\xb5\xd0\xbd
Failed on: b\xd0\xb1i\xd1\x81\xd0\xbe\xd0\xb2\xd0\xbe\xd0\xb3\xd0\xbe
Failed on: b\xd0\xb2\xd0\xb3\xd0\xbe\xd1\x80i
Failed on: b\xd0\xb7\xd0\xb0\xd1\x87i\xd0\xbf\xd0\xb0\xd0\xbb\xd0\xb0
Failed on: b\xd0\xba\xd0\xb8\xd1\x82\xd0\xb0\xd0\xb9\xd1\x81\xd1\x8c\xd0\xbai
Failed on: b1959-\xd0\xb3\xd0\xbe
Failed on: b\xd0\xbf\xd1\x80\xd0\xb8\xd0\xb2i\xd1\x82\xd0\xb0\xd0\xbb\xd0\xb0
Failed on: b\xd0\xbci\xd1\x81\xd1\x8f\xd1\x87\xd0\xbd\xd0\xb0
Failed on: b\xd1\x81\xd0\xbci\xd0\xbb\xd0\xb0
Failed on: b\xd0\xbd\xd0\xb0\xd0\xb4i\xd0\xbb\xd0\xb0
Failed on: b\xd1\x81\xd0\xbe\xd0\xbd\xd0\xbdi
Failed on: b\xd1\x81\xd1\x83\xd1\x85i
Failed on: b\xd0\xbd\x

In [11]:
# # nonsilence_phones.txt
# cut -d ' ' -f 2- $data/local/dict/lexicon.txt | sed 's/ /\n/g' | sort -u > $data/local/dict/nonsilence_phones.txt

In [12]:
# # silence_phones.txt
# echo 'SIL' > $data/local/dict/silence_phones.txt
# echo 'SPN' >> $data/local/dict/silence_phones.txt
# echo 'NSN' >> $data/local/dict/silence_phones.txt

In [13]:
# # optional_silence.txt
# echo 'SIL' > $data/local/dict/optional_silence.txt

In [14]:
# # extend lexicon with silence phones
# echo '!SIL SIL' >> $data/local/dict/lexicon.txt
# echo '<SPOKEN_NOISE> SPN' >> $data/local/dict/lexicon.txt
# echo '<UNK> SPN' >> $data/local/dict/lexicon.txt
# echo '<NOISE> NSN' >> $data/local/dict/lexicon.txt

# Generating Kaldi's lang dir

In [11]:
dvc run \
-d $data/local/dict/lexicon.txt -d $data/local/dict/silence_phones.txt -d $data/local/dict/nonsilence_phones.txt -d $data/local/dict/optional_silence.txt -d $data/local/dict/extra_questions.txt \
-o $data/lang \
-f $UKROASR_ROOT/prepare_lang_dir.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/prepare_lang.sh --position-dependent-phones false $data/local/dict '<UNK>' $data/local/lang $data/lang"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/prepare_lang.sh --position-dependent-phones false /data/exp/kostya/ukroASR/data/local/dict '<UNK>' /data/exp/kostya/ukroASR/data/local/lang /data/exp/kostya/ukroASR/data/lang
utils/prepare_lang.sh --position-dependent-phones false /data/exp/kostya/ukroASR/data/local/dict <UNK> /data/exp/kostya/ukroASR/data/local/lang /data/exp/kostya/ukroASR/data/lang
Checking /data/exp/kostya/ukroASR/data/local/dict/silence_phones.txt ...
--> reading /data/exp/kostya/ukroASR/data/local/dict/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> /data/exp/kostya/ukroASR/data/local/dict/silence_phones.txt is OK

Checking /data/exp/kostya/ukroASR/data/local/dict/optional_silence.txt ...
--> reading /data/exp/kostya/ukroASR/data/local/dict/optional_silence.txt
--> text seems to be UTF-8 or ASCII, checki

--> /data/exp/kostya/ukroASR/data/lang/L_disambig.fst is olabel sorted
--> SUCCESS [validating lang directory /data/exp/kostya/ukroASR/data/lang]
                                                                                
To track the changes with git, run:

	git add prepare_lang_dir.dvc data/.gitignore


# Generating LM

In [11]:
dvc run \
-d $data/train/text \
-o $data/lm_ukr.gz \
"cut -d' ' -f2- $data/train/text | ngram-count -text - -order 3 -gt1min 1 -gt2min 1 -lm - | gzip -c > $data/lm_ukr.gz"

Running command:                                                        
	cut -d' ' -f2- /data/exp/kostya/ukroASR/data/train/text | ngram-count -text - -order 3 -gt1min 1 -gt2min 1 -lm - | gzip -c > /data/exp/kostya/ukroASR/data/lm_ukr.gz
                                                                        
To track the changes with git, run:

	git add data/.gitignore lm_ukr.gz.dvc


In [7]:
dvc run -y \
-d $data/dev/text -d $data/lm_ukr.gz \
-m $exp/metrics/lm.json \
"echo {\'LM perplexity\': $(cut -d' ' -f2- $data/dev/text | ngram -lm $data/lm_ukr.gz -ppl - | tail -1 | awk '{print $6}')} > $exp/metrics/lm.json"

Running command:                                                        
	echo {\'LM perplexity\': 1407.369} > /data/exp/kostya/ukroASR/exp/metrics/lm.json
                                                                        
To track the changes with git, run:

	git add exp/metrics/.gitignore lm.json.dvc


In [8]:
git add exp/metrics/.gitignore lm.json.dvc

# Formatting Kaldi's LM dir

In [14]:
dvc run \
-d $data/lang -d $data/lm_ukr.gz -d $data/local/dict/lexicon.txt \
-o $data/lang_ukr \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/format_lm.sh $data/lang $data/lm_ukr.gz $data/local/dict/lexicon.txt $data/lang_ukr"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/format_lm.sh /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/data/lm_ukr.gz /data/exp/kostya/ukroASR/data/local/dict/lexicon.txt /data/exp/kostya/ukroASR/data/lang_ukr
Converting '/data/exp/kostya/ukroASR/data/lm_ukr.gz' to FST
arpa2fst --disambig-symbol=#0 --read-symbol-table=/data/exp/kostya/ukroASR/data/lang_ukr/words.txt - /data/exp/kostya/ukroASR/data/lang_ukr/G.fst 
LOG (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:94) Reading \data\ section.
LOG (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:149) Reading \1-grams: section.
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 8 [-5.49069	03	-0.06813353] skipped: word '03' not in symbol table
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 9 [-5.49069	05	-0.06018978] skipped: word '05' not in symbol table
WARNING (arpa2fst[5.5.

# Computing features

In [10]:
dvc run -y \
-d $data/train/wav.scp -d $data/train/segments -d $data/train/utt2spk \
-o $exp/train_mfcc -o $data/train/feats.scp \
-f $UKROASR_ROOT/mfcc_train.dvc \
"python3.6 $UKROASR_ROOT/features/make_mfcc.py $data/train $exp/log/mfcc $exp/train_mfcc"

Running command:                                                                
	python3.6 /data/exp/kostya/ukroASR/features/make_mfcc.py /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/exp/log/mfcc /data/exp/kostya/ukroASR/exp/train_mfcc
/bin/sh: 18: ./cmd.sh: [[: not found
steps/make_mfcc.sh /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/exp/log/mfcc /data/exp/kostya/ukroASR/exp/train_mfcc
utils/validate_data_dir.sh: Successfully validated data-directory /data/exp/kostya/ukroASR/data/train
steps/make_mfcc.sh [info]: segments file exists: using that.
Succeeded creating MFCC features for train
                                                                                
To track the changes with git, run:

	git add mfcc_train.dvc exp/.gitignore


In [11]:
dvc run -y \
-d $data/dev/wav.scp -d $data/dev/segments -d $data/dev/utt2spk \
-o $exp/dev_mfcc -o $data/dev/feats.scp \
-f $UKROASR_ROOT/mfcc_dev.dvc \
"python3.6 $UKROASR_ROOT/features/make_mfcc.py $data/dev $exp/log/mfcc $exp/dev_mfcc"

Running command:                                                                
	python3.6 /data/exp/kostya/ukroASR/features/make_mfcc.py /data/exp/kostya/ukroASR/data/dev /data/exp/kostya/ukroASR/exp/log/mfcc /data/exp/kostya/ukroASR/exp/dev_mfcc
/bin/sh: 18: ./cmd.sh: [[: not found
steps/make_mfcc.sh /data/exp/kostya/ukroASR/data/dev /data/exp/kostya/ukroASR/exp/log/mfcc /data/exp/kostya/ukroASR/exp/dev_mfcc
utils/validate_data_dir.sh: Successfully validated data-directory /data/exp/kostya/ukroASR/data/dev
steps/make_mfcc.sh [info]: segments file exists: using that.
Succeeded creating MFCC features for dev
                                                                                
To track the changes with git, run:

	git add mfcc_dev.dvc exp/.gitignore


In [12]:
git add mfcc_train.dvc mfcc_dev.dvc exp/.gitignore

In [13]:
git commit -m "Compute features"

[master 8dfb300] Compute features
 3 files changed, 44 insertions(+)
 create mode 100644 exp/.gitignore
 create mode 100644 mfcc_dev.dvc
 create mode 100644 mfcc_train.dvc


In [23]:
# compute cmvn features

In [8]:
for split in train dev
do
dvc run -y \
-d $data/$split/feats.scp -d $data/$split/utt2spk \
-o $exp/${split}_cmvn -o $data/$split/cmvn.scp \
-f $UKROASR_ROOT/cmvn_${split}.dvc \
"python3.6 $UKROASR_ROOT/features/compute_cmvn_stats.py $data/$split $exp/log/cmvn $exp/${split}_cmvn"
done

Running command:                                                                
	python3.6 /data/exp/kostya/ukroASR/features/compute_cmvn_stats.py /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/exp/log/cmvn /data/exp/kostya/ukroASR/exp/train_cmvn
/bin/sh: 18: ./cmd.sh: [[: not found
steps/compute_cmvn_stats.sh /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/exp/log/cmvn /data/exp/kostya/ukroASR/exp/train_cmvn
Succeeded creating CMVN stats for train
                                                                                
To track the changes with git, run:

	git add cmvn_train.dvc
Running command:                                                                
	python3.6 /data/exp/kostya/ukroASR/features/compute_cmvn_stats.py /data/exp/kostya/ukroASR/data/dev /data/exp/kostya/ukroASR/exp/log/cmvn /data/exp/kostya/ukroASR/exp/dev_cmvn
/bin/sh: 18: ./cmd.sh: [[: not found
steps/compute_cmvn_stats.sh /data/exp/kostya/ukroASR/data/dev /data/exp/kostya/ukr

: 1

In [9]:
git add cmvn_dev.dvc cmvn_train.dvc

In [10]:
git commit -m "Compute cmvn stats"

[master 1667353] Compute cmvn stats
 2 files changed, 38 insertions(+)
 create mode 100644 cmvn_dev.dvc
 create mode 100644 cmvn_train.dvc


# Sampling again

In [14]:
dvc run -y \
-d $data/train/text -d $data/train/utt2spk -d $data/train/segments -d $data/train/spk2utt -d $data/train/wav.scp -d $data/train/feats.scp -d $data/train/cmvn.scp \
-o $data/train_2kshort/text -o $data/train_2kshort/utt2spk -o $data/train_2kshort/segments -o $data/train_2kshort/spk2utt -o $data/train_2kshort/wav.scp -o $data/train_2kshort/feats.scp -o $data/train_2kshort/cmvn.scp \
-f $UKROASR_ROOT/train_2kshort_data.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh --shortest $data/train 2000 $data/train_2kshort"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh --shortest /data/exp/kostya/ukroASR/data/train 2000 /data/exp/kostya/ukroASR/data/train_2kshort
feat-to-len scp:/data/exp/kostya/ukroASR/data/train/feats.scp ark,t:/data/exp/kostya/ukroASR/data/train_2kshort/tmp.len 
utils/subset_data_dir.sh: reducing #utt from 10811 to 2000
                                                                        
To track the changes with git, run:

	git add data/train_2kshort/.gitignore train_2kshort_data.dvc


In [15]:
dvc run -y \
-d $data/train/text -d $data/train/utt2spk -d $data/train/segments -d $data/train/spk2utt -d $data/train/wav.scp -d $data/train/feats.scp -d $data/train/cmvn.scp \
-o $data/train_5k/text -o $data/train_5k/utt2spk -o $data/train_5k/segments -o $data/train_5k/spk2utt -o $data/train_5k/wav.scp -o $data/train_5k/feats.scp -o $data/train_5k/cmvn.scp \
-f $UKROASR_ROOT/train_5k_data.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh $data/train 5000 $data/train_5k"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh /data/exp/kostya/ukroASR/data/train 5000 /data/exp/kostya/ukroASR/data/train_5k
utils/subset_data_dir.sh: reducing #utt from 10811 to 5000
                                                                        
To track the changes with git, run:

	git add data/train_5k/.gitignore train_5k_data.dvc


In [17]:
dvc run -y \
-d $data/dev/text -d $data/dev/utt2spk -d $data/dev/segments -d $data/dev/spk2utt -d $data/dev/wav.scp -d $data/dev/feats.scp -d $data/dev/cmvn.scp \
-o $data/dev_0.75kshort/text -o $data/dev_0.75kshort/utt2spk -o $data/dev_0.75kshort/segments -o $data/dev_0.75kshort/spk2utt -o $data/dev_0.75kshort/wav.scp -o $data/dev_0.75kshort/feats.scp -o $data/dev_0.75kshort/cmvn.scp \
-f $UKROASR_ROOT/dev_0.75kshort_data.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh --shortest $data/dev 750 $data/dev_0.75kshort"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh --shortest /data/exp/kostya/ukroASR/data/dev 750 /data/exp/kostya/ukroASR/data/dev_0.75kshort
feat-to-len scp:/data/exp/kostya/ukroASR/data/dev/feats.scp ark,t:/data/exp/kostya/ukroASR/data/dev_0.75kshort/tmp.len 
utils/subset_data_dir.sh: reducing #utt from 879 to 750
                                                                        
To track the changes with git, run:

	git add dev_0.75kshort_data.dvc


In [18]:
dvc run -y \
-d $data/dev_0.75kshort/text -d $data/dev_0.75kshort/utt2spk -d $data/dev_0.75kshort/segments -d $data/dev_0.75kshort/spk2utt -d $data/dev_0.75kshort/wav.scp -d $data/dev_0.75kshort/feats.scp -d $data/dev_0.75kshort/cmvn.scp \
-o $data/dev_0.15k/text -o $data/dev_0.15k/utt2spk -o $data/dev_0.15k/segments -o $data/dev_0.15k/spk2utt -o $data/dev_0.15k/wav.scp -o $data/dev_0.15k/feats.scp -o $data/dev_0.15k/cmvn.scp \
-f $UKROASR_ROOT/dev_0.15k_data.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh $data/dev_0.75kshort 150 $data/dev_0.15k"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/subset_data_dir.sh /data/exp/kostya/ukroASR/data/dev_0.75kshort 150 /data/exp/kostya/ukroASR/data/dev_0.15k
utils/subset_data_dir.sh: reducing #utt from 750 to 150
                                                                        
To track the changes with git, run:

	git add dev_0.15k_data.dvc data/dev_0.15k/.gitignore


In [20]:
git add data/train_2kshort/.gitignore train_2kshort_data.dvc data/train_5k/.gitignore train_5k_data.dvc dev_0.75kshort_data.dvc data/dev_0.75kshort/.gitignore dev_0.15k_data.dvc data/dev_0.15k/.gitignore

In [21]:
git commit -m "Add subsets of train and dev"

[master 9990b5f] Add subsets of train and dev
 8 files changed, 244 insertions(+)
 create mode 100644 data/dev_0.15k/.gitignore
 create mode 100644 data/dev_0.75kshort/.gitignore
 create mode 100644 data/train_2kshort/.gitignore
 create mode 100644 data/train_5k/.gitignore
 create mode 100644 dev_0.15k_data.dvc
 create mode 100644 dev_0.75kshort_data.dvc
 create mode 100644 train_2kshort_data.dvc
 create mode 100644 train_5k_data.dvc


# Training and alignment

# monophone

In [77]:
mkdir $exp/mono

In [78]:
dvc run -y \
-d $data/train_2kshort/feats.scp -d $data/train_2kshort/utt2spk -d $data/train_2kshort/text -d $data/train_2kshort/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones/sets.int -d $data/lang/phones/disambig.int -d $data/lang/phones/optional_silence.csl \
-o $exp/mono/final.mdl -o $exp/mono/tree -o $exp/mono/phones.txt \
-w $exp/mono \
-f $UKROASR_ROOT/mono.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_mono.sh --boost-silence 1.25 --nj 8 $data/train_2kshort $data/lang $exp/mono"


Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5/../../../egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_mono.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_2kshort /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono
steps/train_mono.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_2kshort /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning dat

In [39]:
git add mono.dvc exp/mono/.gitignore

In [43]:
dvc run -y \
-d $exp/mono/final.mdl -d $exp/mono/tree -d $data/lang_ukr/L.fst -d $data/lang_ukr/G.fst -d $data/lang_ukr/phones.txt -d $data/lang_ukr/words.txt -d $data/lang_ukr/phones/silence.csl -d $data/lang_ukr/phones/disambig.int \
-o $exp/mono/graph/HCLG.fst -o $exp/mono/graph/words.txt \
-f $UKROASR_ROOT/mono_graph.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh $data/lang_ukr $exp/mono $exp/mono/graph"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang_ukr /data/exp/kostya/ukroASR/exp/mono /data/exp/kostya/ukroASR/exp/mono/graph
tree-info /data/exp/kostya/ukroASR/exp/mono/tree 
tree-info /data/exp/kostya/ukroASR/exp/mono/tree 
make-h-transducer --disambig-syms-out=/data/exp/kostya/ukroASR/exp/mono/graph/disambig_tid.int --transition-scale=1.0 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_1_0 /data/exp/kostya/ukroASR/exp/mono/tree /data/exp/kostya/ukroASR/exp/mono/final.mdl 
fstminimizeencoded 
fstrmsymbols /data/exp/kostya/ukroASR/exp/mono/graph/disambig_tid.int 
fstdeterminizestar --use-log=true 
fstrmepslocal 
fsttablecompose /data/exp/kostya/ukroASR/exp/mono/graph/Ha.fst /data/exp/kostya/ukroASR/data/lang_ukr/tmp/CLG_1_0.fst 
fstisstochastic /data/exp/kostya/ukroASR/exp/mono/graph/HCLGa.fst 
0.000424194 -0.0706752
HCLGa is not stochastic
add-self

In [44]:
git add mono_graph.dvc exp/mono/graph/.gitignore

In [102]:
dvc run -y \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/mono/final.mdl -d $exp/mono/graph/HCLG.fst -d $exp/mono/graph/words.txt \
-m $exp/metrics/mono.json \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 $exp/mono/graph $data/dev_0.15k $exp/mono/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/mono/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/mono.json"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5/../../../egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/mono/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/mono.json
steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/mono/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k
decode.sh: feature type is delta
steps/diagnostic/analyze_lats.sh --cmd run.pl /data/exp/kostya/ukroASR/exp/mono/graph /data/exp/kostya/ukroASR/exp/mono/decode_dev_0.15k
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 50.6666666667% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_lats.sh: see stats in /data/exp/ko

In [103]:
git add mono.json.dvc

In [8]:
cat exp/metrics/mono.json

{'WER': 72.83}


In [17]:
#base %WER 70.18
#2kshort %WER 75.96
#2kshort  + correct spk _2 %WER 75.41
#2kshort  + correct spk _2 no _I_B_S_E %WER 76.24
#2kshort  + correct spk _2 no _I_B_S_E small dev %WER 74.72 
#2kshort  + correct spk _2 no _I_B_S_E shared silence phones small dev %WER 75.25
#2kshort  + correct spk _2 no _I_B_S_E extra_questions stress %WER 75.70
######################################################################################################################
#2kshort  + 8 spk + no IBSE %WER 73.65
#2kshort  + 8 spk + no IBSE %WER extra_questions stress 74.34

# triphone

In [ ]:
# aligning train data

In [16]:
dvc run -y \
-d $data/train_5k/feats.scp -d $data/train_5k/utt2spk -d $data/train_5k/text -d $data/train_5k/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones/disambig.int -d $data/lang/phones/optional_silence.csl -d $data/lang/phones.txt -d $data/lang/oov.int -d $exp/mono/final.mdl -d $exp/mono/tree -d $exp/mono/phones.txt \
$(for f in $exp/mono_train_5k_ali/ali.*.gz; do echo -o $f; done) \
-f $UKROASR_ROOT/mono_ali.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --boost-silence 1.25 --nj 8 $data/train_5k $data/lang $exp/mono $exp/mono_train_5k_ali"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
steps/align_si.sh --boost-silence 1.25 --nj 8 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
steps/align_si.sh: feature type is delta
steps/align_si.sh: aligning data in /data/exp/kostya/ukroASR/data/train_5k using model from /data/exp/kostya/ukroASR/exp/mono, putting alignments in /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono_train_5k_ali
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 49.7093605933% of the 

In [42]:
git add exp/.gitignore mono_ali.dvc

In [21]:
dvc run -y \
$(for f in $exp/mono_train_5k_ali/ali.*.gz; do echo -d $f; done) -d $exp/mono_train_5k_ali/phones.txt -d $exp/mono_train_5k_ali/num_jobs -d $exp/mono_train_5k_ali/final.mdl -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones/sets.int -d $data/lang/phones/roots.int -d $data/lang/phones/disambig.int -d $data/lang/phones/optional_silence.csl -d $data/lang/phones/extra_questions.int -d $data/train_5k/feats.scp -d $data/train_5k/utt2spk -d $data/train_5k/cmvn.scp \
-o $exp/tri1/final.mdl -o $exp/tri1/tree -o $exp/tri1/phones.txt \
-f $UKROASR_ROOT/tri1.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_deltas.sh --boost-silence 1.25 2000 10000 $data/train_5k $data/lang $exp/mono_train_5k_ali $exp/tri1; \
cp $exp/tri1/final.mdl $exp/tri1/final.mdl_copy; mv $exp/tri1/final.mdl_copy $exp/tri1/final.mdl"


Running command:
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_deltas.sh --boost-silence 1.25 2000 10000 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono_train_5k_ali /data/exp/kostya/ukroASR/exp/tri1; cp /data/exp/kostya/ukroASR/exp/tri1/final.mdl /data/exp/kostya/ukroASR/exp/tri1/final.mdl_copy; mv /data/exp/kostya/ukroASR/exp/tri1/final.mdl_copy /data/exp/kostya/ukroASR/exp/tri1/final.mdl
steps/train_deltas.sh --boost-silence 1.25 2000 10000 /data/exp/kostya/ukroASR/data/train_5k /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/mono_train_5k_ali /data/exp/kostya/ukroASR/exp/tri1
steps/train_deltas.sh: accumulating tree stats
steps/train_deltas.sh: getting questions for tree-building, via clustering
steps/train_deltas.sh: building the tree
WARNING (gmm-init-model[5.5.303~1-5a34a]:InitAmGmm():gmm-init-model.cc:55) Tree has pdf-id 2 with no stats; corresponding phone list: 3 
WARNING (gmm-

In [46]:
git add tri1.dvc

In [47]:
dvc run -y \
-d $exp/tri1/final.mdl -d $exp/tri1/tree -d $data/lang_ukr/L.fst -d $data/lang_ukr/G.fst -d $data/lang_ukr/phones.txt -d $data/lang_ukr/words.txt -d $data/lang_ukr/phones/silence.csl -d $data/lang_ukr/phones/disambig.int \
-o $exp/tri1/graph/HCLG.fst -o $exp/tri1/graph/words.txt \
-f $UKROASR_ROOT/tri1_graph.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh $data/lang_ukr $exp/tri1 $exp/tri1/graph"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang_ukr /data/exp/kostya/ukroASR/exp/tri1 /data/exp/kostya/ukroASR/exp/tri1/graph
tree-info /data/exp/kostya/ukroASR/exp/tri1/tree 
tree-info /data/exp/kostya/ukroASR/exp/tri1/tree 
fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=/data/exp/kostya/ukroASR/data/lang_ukr/phones/disambig.int --write-disambig-syms=/data/exp/kostya/ukroASR/data/lang_ukr/tmp/disambig_ilabels_3_1.int /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_3_1.30415 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/LG.fst 
fstisstochastic /data/exp/kostya/ukroASR/data/lang_ukr/tmp/CLG_3_1.fst 
0 -0.0361818
[info]: CLG not stochastic.
make-h-transducer --disambig-syms-out=/data/exp/kostya/ukroASR/exp/tri1/graph/disambig_tid.int --transition-scale=1.0 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_3_1 /data/exp/kosty

In [48]:
git add tri1_graph.dvc exp/tri1/graph/.gitignore

In [49]:
dvc run -y \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri1/final.mdl -d $exp/tri1/graph/HCLG.fst -d $exp/tri1/graph/words.txt \
-m $exp/metrics/tri1.json \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 $exp/tri1/graph $data/dev_0.15k $exp/tri1/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri1/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri1.json"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri1/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/tri1.json
steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri1/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k
decode.sh: feature type is delta
steps/diagnostic/analyze_lats.sh --cmd run.pl /data/exp/kostya/ukroASR/exp/tri1/graph /data/exp/kostya/ukroASR/exp/tri1/decode_dev_0.15k
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 51.3333333333% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_lats.sh: see stats in /data/exp/kostya/ukroASR/exp/tri

In [50]:
git add tri1.json.dvc exp/metrics/.gitignore

In [51]:
cat exp/metrics/tri1.json

{'WER': 56.63}


In [ ]:
# base %WER 60.95
# half + 2kshort %WER 61.70
# whole_train + 2kshort %WER 61.73
# half + 2kshort + correct spk_2 no _I_B_S_E short dev %WER 61.26
# half + 2kshort + correct spk_2 no _I_B_S_E shared silence phones %WER 63.56
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 61.51
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 56.72
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 56.28

# lda mllt

In [38]:
dvc run -y \
-d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/text -d $data/train/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones/disambig.int -d $data/lang/phones/optional_silence.csl -d $data/lang/phones.txt -d $data/lang/oov.int -d $exp/tri1/final.mdl -d $exp/tri1/tree -d $exp/tri1/phones.txt \
$(for f in $exp/tri1_train_ali/ali.*.gz; do echo -o $f; done) \
-f $UKROASR_ROOT/tri_ali.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --nj 8 $data/train $data/lang $exp/tri1 $exp/tri1_train_ali"


Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1 /data/exp/kostya/ukroASR/exp/tri1_train_ali
steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1 /data/exp/kostya/ukroASR/exp/tri1_train_ali
steps/align_si.sh: feature type is delta
steps/align_si.sh: aligning data in /data/exp/kostya/ukroASR/data/train using model from /data/exp/kostya/ukroASR/exp/tri1, putting alignments in /data/exp/kostya/ukroASR/exp/tri1_train_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1_train_ali
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 49.6139893963% of the time at utterance end.  This may not be optimal.
steps/diagnost

In [20]:
git add tri_ali.dvc

In [23]:
mkdir /data/exp/kostya/ukroASR/exp/tri2b

In [30]:
dvc run -y \
$(for f in $exp/tri1_train_ali/ali.*.gz; do echo -d $f; done) -d $exp/tri1_train_ali/phones.txt -d $exp/tri1_train_ali/num_jobs -d $exp/tri1_train_ali/final.mdl -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones/sets.int -d $data/lang/phones/roots.int -d $data/lang/phones/disambig.int -d $data/lang/phones/optional_silence.csl -d $data/lang/phones/extra_questions.int -d $data/lang/phones/context_indep.csl -d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/cmvn.scp \
-o $exp/tri2b/final.mdl -o $exp/tri2b/final.mat -o $exp/tri2b/tree -o $exp/tri2b/phones.txt \
-f $UKROASR_ROOT/tri2b.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_lda_mllt.sh --splice-opts \"--left-context=3 --right-context=3\" 2500 15000 $data/train $data/lang $exp/tri1_train_ali $exp/tri2b; \
cp $exp/tri2b/final.mdl $exp/tri2b/final.mdl_copy; mv $exp/tri2b/final.mdl_copy $exp/tri2b/final.mdl; \
cp $exp/tri2b/final.mat $exp/tri2b/final.mat_copy; mv $exp/tri2b/final.mat_copy $exp/tri2b/final.mat"


Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_lda_mllt.sh --splice-opts "--left-context=3 --right-context=3" 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1_train_ali /data/exp/kostya/ukroASR/exp/tri2b; cp /data/exp/kostya/ukroASR/exp/tri2b/final.mdl /data/exp/kostya/ukroASR/exp/tri2b/final.mdl_copy; mv /data/exp/kostya/ukroASR/exp/tri2b/final.mdl_copy /data/exp/kostya/ukroASR/exp/tri2b/final.mdl; cp /data/exp/kostya/ukroASR/exp/tri2b/final.mat /data/exp/kostya/ukroASR/exp/tri2b/final.mat_copy; mv /data/exp/kostya/ukroASR/exp/tri2b/final.mat_copy /data/exp/kostya/ukroASR/exp/tri2b/final.mat
steps/train_lda_mllt.sh --splice-opts --left-context=3 --right-context=3 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri1_train_ali /data/exp/kostya/ukroASR/exp/tri2b
ste

In [31]:
git add exp/tri2b/.gitignore tri2b.dvc

In [32]:
dvc run -y \
-d $exp/tri2b/final.mdl -d $exp/tri2b/tree -d $data/lang_ukr/L.fst -d $data/lang_ukr/G.fst -d $data/lang_ukr/phones.txt -d $data/lang_ukr/words.txt -d $data/lang_ukr/phones/silence.csl -d $data/lang_ukr/phones/disambig.int \
-o $exp/tri2b/graph/HCLG.fst -o $exp/tri2b/graph/words.txt \
-f $UKROASR_ROOT/tri2b_graph.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh $data/lang_ukr $exp/tri2b $exp/tri2b/graph"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang_ukr /data/exp/kostya/ukroASR/exp/tri2b /data/exp/kostya/ukroASR/exp/tri2b/graph
tree-info /data/exp/kostya/ukroASR/exp/tri2b/tree 
tree-info /data/exp/kostya/ukroASR/exp/tri2b/tree 
make-h-transducer --disambig-syms-out=/data/exp/kostya/ukroASR/exp/tri2b/graph/disambig_tid.int --transition-scale=1.0 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_3_1 /data/exp/kostya/ukroASR/exp/tri2b/tree /data/exp/kostya/ukroASR/exp/tri2b/final.mdl 
fsttablecompose /data/exp/kostya/ukroASR/exp/tri2b/graph/Ha.fst /data/exp/kostya/ukroASR/data/lang_ukr/tmp/CLG_3_1.fst 
fstrmepslocal 
fstrmsymbols /data/exp/kostya/ukroASR/exp/tri2b/graph/disambig_tid.int 
fstdeterminizestar --use-log=true 
fstminimizeencoded 
fstisstochastic /data/exp/kostya/ukroASR/exp/tri2b/graph/HCLGa.fst 
0.604139 -0.102449
HCLGa is not stochastic
ad

In [33]:
git add tri2b_graph.dvc exp/tri2b/graph/.gitignore

In [34]:
dvc run -y \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri2b/final.mdl -d $exp/tri2b/final.mat -d $exp/tri2b/graph/HCLG.fst -d $exp/tri2b/graph/words.txt \
-m $exp/metrics/tri2b.json \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 $exp/tri2b/graph $data/dev_0.15k $exp/tri2b/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri2b/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri2b.json"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri2b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/tri2b.json
steps/decode.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri2b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k
decode.sh: feature type is lda
steps/diagnostic/analyze_lats.sh --cmd run.pl /data/exp/kostya/ukroASR/exp/tri2b/graph /data/exp/kostya/ukroASR/exp/tri2b/decode_dev_0.15k
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 49.3333333333% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_lats.sh: see stats in /data/exp/kostya/ukroASR/e

In [35]:
git add tri2b.json.dvc exp/metrics/.gitignore

In [36]:
cat $exp/metrics/tri2b.json

{'WER': 55.49}


In [ ]:
#base %WER 68.48
# half + 2kshort %WER 68.22
# full + 2kshort + correct spk %WER 67.81
# full + 2kshort + correct spk _2 %WER 65.25
# 2kshort + half + correct spk_2 + no _I_B_S_E %WER 64.77
# 2kshort + half + correct spk_2 + no _I_B_S_E shared silence phones  %WER 67.73
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 64.91
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 56.31
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 57.05

# LDA + MLLT + SAT

In [43]:
dvc run -y \
-d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/text -d $data/train/cmvn.scp -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones/disambig.int -d $data/lang/phones/optional_silence.csl -d $data/lang/phones.txt -d $data/lang/oov.int -d $exp/tri2b/final.mdl -d $exp/tri2b/final.mat -d $exp/tri2b/full.mat -d $exp/tri2b/tree -d $exp/tri2b/phones.txt \
$(for f in $exp/tri2b_train_ali/ali.*.gz; do echo -o $f; done) -o $exp/tri2b_train_ali/phones.txt -o $exp/tri2b_train_ali/num_jobs -o $exp/tri2b_train_ali/final.mdl \
-f $UKROASR_ROOT/tri2b_ali.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --nj 8 $data/train $data/lang $exp/tri2b $exp/tri2b_train_ali"


Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b /data/exp/kostya/ukroASR/exp/tri2b_train_ali
steps/align_si.sh --nj 8 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b /data/exp/kostya/ukroASR/exp/tri2b_train_ali
steps/align_si.sh: feature type is lda
steps/align_si.sh: aligning data in /data/exp/kostya/ukroASR/data/train using model from /data/exp/kostya/ukroASR/exp/tri2b, putting alignments in /data/exp/kostya/ukroASR/exp/tri2b_train_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b_train_ali
analyze_phone_length_stats.py: WARNING: optional-silence SIL is seen only 46.0542252865% of the time at utterance end.  This may not be optimal.
steps/dia

In [44]:
git add exp/tri2b_train_ali/.gitignore tri2b_ali.dvc

In [48]:
dvc run -y \
$(for f in $exp/tri2b_train_ali/ali.*.gz; do echo -d $f; done) -d $exp/tri2b_train_ali/phones.txt -d $exp/tri2b_train_ali/num_jobs -d $exp/tri2b_train_ali/final.mdl -d $data/lang/L.fst -d $data/lang/words.txt -d $data/lang/phones.txt -d $data/lang/topo -d $data/lang/phones/sets.int -d $data/lang/phones/roots.int -d $data/lang/phones/disambig.int -d $data/lang/phones/optional_silence.csl -d $data/lang/phones/extra_questions.int -d $data/lang/phones/context_indep.csl -d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/cmvn.scp \
-o $exp/tri3b/final.mdl -o $exp/tri3b/final.mat -o $exp/tri3b/tree -o $exp/tri3b/phones.txt \
-f $UKROASR_ROOT/tri3b.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_sat.sh 2500 15000 $data/train $data/lang $exp/tri2b_train_ali $exp/tri3b; \
cp $exp/tri3b/final.mdl $exp/tri3b/final.mdl_copy; mv $exp/tri3b/final.mdl_copy $exp/tri3b/final.mdl; \
cp $exp/tri3b/final.mat $exp/tri3b/final.mat_copy; mv $exp/tri3b/final.mat_copy $exp/tri3b/final.mat"


Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/train_sat.sh 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b_train_ali /data/exp/kostya/ukroASR/exp/tri3b; cp /data/exp/kostya/ukroASR/exp/tri3b/final.mdl /data/exp/kostya/ukroASR/exp/tri3b/final.mdl_copy; mv /data/exp/kostya/ukroASR/exp/tri3b/final.mdl_copy /data/exp/kostya/ukroASR/exp/tri3b/final.mdl; cp /data/exp/kostya/ukroASR/exp/tri3b/final.mat /data/exp/kostya/ukroASR/exp/tri3b/final.mat_copy; mv /data/exp/kostya/ukroASR/exp/tri3b/final.mat_copy /data/exp/kostya/ukroASR/exp/tri3b/final.mat
steps/train_sat.sh 2500 15000 /data/exp/kostya/ukroASR/data/train /data/exp/kostya/ukroASR/data/lang /data/exp/kostya/ukroASR/exp/tri2b_train_ali /data/exp/kostya/ukroASR/exp/tri3b
steps/train_sat.sh: feature type is lda
steps/train_sat.sh: obtaining initial fMLLR transforms since not presen

In [49]:
git add tri3b.dvc

In [50]:
dvc run -y \
-d $exp/tri3b/final.mdl -d $exp/tri3b/tree -d $data/lang_ukr/L.fst -d $data/lang_ukr/G.fst -d $data/lang_ukr/phones.txt -d $data/lang_ukr/words.txt -d $data/lang_ukr/phones/silence.csl -d $data/lang_ukr/phones/disambig.int \
-o $exp/tri3b/graph/HCLG.fst -o $exp/tri3b/graph/words.txt \
-f $UKROASR_ROOT/tri3b_graph.dvc \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh $data/lang_ukr $exp/tri3b $exp/tri3b/graph"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; utils/mkgraph.sh /data/exp/kostya/ukroASR/data/lang_ukr /data/exp/kostya/ukroASR/exp/tri3b /data/exp/kostya/ukroASR/exp/tri3b/graph
tree-info /data/exp/kostya/ukroASR/exp/tri3b/tree 
tree-info /data/exp/kostya/ukroASR/exp/tri3b/tree 
make-h-transducer --disambig-syms-out=/data/exp/kostya/ukroASR/exp/tri3b/graph/disambig_tid.int --transition-scale=1.0 /data/exp/kostya/ukroASR/data/lang_ukr/tmp/ilabels_3_1 /data/exp/kostya/ukroASR/exp/tri3b/tree /data/exp/kostya/ukroASR/exp/tri3b/final.mdl 
fsttablecompose /data/exp/kostya/ukroASR/exp/tri3b/graph/Ha.fst /data/exp/kostya/ukroASR/data/lang_ukr/tmp/CLG_3_1.fst 
fstrmepslocal 
fstrmsymbols /data/exp/kostya/ukroASR/exp/tri3b/graph/disambig_tid.int 
fstminimizeencoded 
fstdeterminizestar --use-log=true 
fstisstochastic /data/exp/kostya/ukroASR/exp/tri3b/graph/HCLGa.fst 
0.604139 -0.102441
HCLGa is not stochastic
ad

In [51]:
git add tri3b_graph.dvc exp/tri3b/graph/.gitignore

In [52]:
dvc run -y \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri3b/final.mdl -d $exp/tri3b/tree -d $exp/tri3b/final.mat -d $exp/tri3b/graph/HCLG.fst -d $exp/tri3b/graph/words.txt -d $exp/tri3b/graph/phones/silence.csl \
-m $exp/metrics/tri3b.json \
"cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode_fmllr.sh --nj 4 $exp/tri3b/graph $data/dev_0.15k $exp/tri3b/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri3b/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri3b.json"

Running command:                                                        
	cd /data/kaldi/prod/egs/wsj/s5; . ./path.sh; . ./cmd.sh; steps/decode_fmllr.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri3b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k; /data/exp/kostya/ukroASR/parce_wer_report.sh /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k/scoring_kaldi/best_wer /data/exp/kostya/ukroASR/exp/metrics/tri3b.json
steps/decode_fmllr.sh --nj 4 /data/exp/kostya/ukroASR/exp/tri3b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k
steps/decode.sh --scoring-opts  --num-threads 1 --skip-scoring false --acwt 0.083333 --nj 4 --cmd run.pl --beam 10.0 --model /data/exp/kostya/ukroASR/exp/tri3b/final.alimdl --max-active 2000 /data/exp/kostya/ukroASR/exp/tri3b/graph /data/exp/kostya/ukroASR/data/dev_0.15k /data/exp/kostya/ukroASR/exp/tri3b/decode_dev_0.15k.si
decode.sh: feature type is lda
steps/diagnostic/a

In [53]:
git add tri3b.json.dvc exp/metrics/.gitignore

In [54]:
cat $exp/metrics/tri3b.json

{'WER': 45.75}


In [ ]:
# base %WER 54.14
# half + 2kshort %WER 53.50
# 2kshort + half + correct spk_2 + no _I_B_S_E %WER 50.55
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 50.85
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 47.46
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 46.80

In [ ]:
dvc repro tri3b.json.dvc

# Generating LM

In [8]:
cut -d' ' -f2- $data/train/text | \
ngram-count -text - -order 3 -gt1min 1 -gt2min 1 -lm - | gzip -c > $data/lm/lm_ukr.gz

In [9]:
# cut -d' ' -f2- $data/train/text | \
# ngram-count -text - -order 2 -gt1min 1 -gt2min 1 -lm $data/lm/lm_ukr.txt

In [10]:
utils/format_lm.sh $data/lang $data/lm/lm_ukr.gz $data/local/dict/lexicon.txt $data/lang_ukr

Converting '/data/exp/kostya/ukr_asr/data/lm/lm_ukr.gz' to FST
arpa2fst --disambig-symbol=#0 --read-symbol-table=/data/exp/kostya/ukr_asr/data/lang_ukr/words.txt - /data/exp/kostya/ukr_asr/data/lang_ukr/G.fst 
LOG (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:94) Reading \data\ section.
LOG (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:149) Reading \1-grams: section.
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 7 [-5.490523	03	-0.06809548] skipped: word '03' not in symbol table
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 8 [-5.490523	05	-0.06014865] skipped: word '05' not in symbol table
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 9 [-5.490523	06	-0.06809689] skipped: word '06' not in symbol table
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-parser.cc:219) line 10 [-5.490523	1	-0.0593989] skipped: word '1' not in symbol table
WARNING (arpa2fst[5.5.303~1-5a34a]:Read():arpa-file-pars

In [11]:
l $data/lang_ukr

total 29M
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 11 14:15 tmp
-rw-rw-r-- 1 ubuntu ubuntu  614 May 11 18:48 phones.txt
-rw-rw-r-- 1 ubuntu ubuntu 1.1M May 11 18:48 words.txt
-rw-rw-r-- 1 ubuntu ubuntu 1.2K May 11 18:48 topo
-rw-rw-r-- 1 ubuntu ubuntu  12M May 11 18:48 L.fst
-rw-rw-r-- 1 ubuntu ubuntu  12M May 11 18:48 L_disambig.fst
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 11 18:48 phones
-rw-rw-r-- 1 ubuntu ubuntu    2 May 11 18:48 oov.int
-rw-rw-r-- 1 ubuntu ubuntu    6 May 11 18:48 oov.txt
-rw-rw-r-- 1 ubuntu ubuntu 5.1M May 11 18:48 G.fst


In [ ]:
# base - %WER 98.63
# with kaydishi - %WER 78.11
# with nestyako_1 - %WER  60.53
# with nestyako_2 - %WER  58.11
# with shevchuk jinka- - %WER  56.35 - new dev %WER 60.73
# with bigger lm - %WER 56.6 - new dev %WER 60.92
# with povii - new dev %WER 54.14

In [ ]:
# Error analisys

In [ ]:
steps/align_si.sh --nj 8 --cmd "$train_cmd" \
$data/dev $data/lang $data/exp/tri3b $data/exp/tri3b_dev_ali

In [ ]:
grep 'Did not successfully decode file' /data/exp/kostya/ukr_asr/data/exp/tri3b_dev_ali/log/align.*.log

In [ ]:
grep 'Retrying utterance' /data/exp/kostya/ukr_asr/data/exp/tri3b_dev_ali/log/align.*.log

In [ ]:
echo $data/exp/tri3b/decode_dev/scoring_kaldi/wer_details/per_utt

In [ ]:
l $data/exp/tri3b/graph

In [ ]:
echo $data/exp/tri3b/graph/words.txt

In [ ]:
wc $data/local/dict/vocab

In [ ]:
rm -rf $data/local

In [ ]:
rm -rf $data/train_45k

In [ ]:
echo steps/train_mono.sh --boost-silence 1.25 --nj 5 --cmd "$train_cmd" \
$data/train_2kshort $data/lang $data/exp/mono

In [ ]:
l /data/exp/kostya/ukr_asr/data/exp/mono

In [ ]:
cat /data/exp/kostya/ukr_asr/data/exp/mono/log/init.log

In [ ]:
echo steps/train_mono.sh --boost-silence 1.25 --nj 5 --cmd "$train_cmd" \
$data/train_2kshort $data/lang $data/exp/mono

In [ ]:
# debug

In [ ]:
for p in 2 3 4 5 6 7 8 9 11 12 13 14 15 23 31 33 35 43 44 45 47 48 49 51 55 59 60 61 63 64 67 68 69 71 75 79 80 81 83 85 87 91 99 103 104 107 111 113 115 117 119 123 127 135 139 147 149 151 153 155 156 157 159 163 165 167 168 169 171 175 176 177 179 183 185 187 191 193 195 196 197 199 200 201 203 208 209 211 215 216 217 219 223 225 227 231 232 233 235 239 243 245 247 248 249 251 259 260 261 263 265 267 271 272 273 275 279 280 281 283 287 288 289 291 295 299 300 301 303 311 313 315 319 327 329 331 341 343 351 352 353 355 357 359 363 364 365 367
do 
grep $p $data/lang/phones.txt
done


In [ ]:
cat $data/lang/phones.txt

In [ ]:
l $data

In [ ]:
rm -rf $data/local/dict